In [2]:
import pandas as pd
import eurostat

In [12]:
df = eurostat.get_data_df(code="demo_pjan", flags=False)
df.head()

,unit,age,sex,geo\time,2021,2020,2019,2018,2017,2016,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
0,NR,TOTAL,F,AD,NaN,NaN,37388.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NR,TOTAL,F,AL,1419759.0,1425342.0,1432833.0,1431715.0,1423050.0,1417141.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NR,TOTAL,F,AM,1565144.0,1562689.0,1563538.0,1564533.0,1567380.0,1569535.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NR,TOTAL,F,AT,4535712.0,4522292.0,4501742.0,4483749.0,4460424.0,4427918.0,...,3932691.0,3922359.0,3899799.0,3876559.0,3857760.0,3836415.0,3814191.0,3794130.0,3773097.0,3757167.0
4,NR,TOTAL,F,AZ,5065288.0,5039100.0,4999053.0,4960058.0,4918771.0,4870002.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
def _estat_pp_cols(df: pd.DataFrame) -> pd.DataFrame:
    tmp = df.copy()
    for i in tmp.columns:
        if i not in (
            "age",
            "sex",
            "unit",
            "geo",
            "geo\\time",
            "icd10",
            "resid",
            "indic_de",
            "isced11",
            "marsta",
        ):
            tmp[i] = tmp[i].fillna(0)
            tmp[i] = tmp[i].astype(int)
    tmp.rename(columns={"geo\\time": "geo"}, inplace=True)
    return tmp

In [25]:
df2 = _estat_pp_cols(df)
df2

,unit,age,sex,geo,2021,2020,2019,2018,2017,2016,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
0,NR,TOTAL,F,AD,0,0,37388,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NR,TOTAL,F,AL,1419759,1425342,1432833,1431715,1423050,1417141,...,0,0,0,0,0,0,0,0,0,0
2,NR,TOTAL,F,AM,1565144,1562689,1563538,1564533,1567380,1569535,...,0,0,0,0,0,0,0,0,0,0
3,NR,TOTAL,F,AT,4535712,4522292,4501742,4483749,4460424,4427918,...,3932691,3922359,3899799,3876559,3857760,3836415,3814191,3794130,3773097,3757167
4,NR,TOTAL,F,AZ,5065288,5039100,4999053,4960058,4918771,4870002,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17524,NR,Y_OPEN,T,SM,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
17525,NR,Y_OPEN,T,TR,5780,5567,174875,5416,145341,127986,...,97602,90125,83222,76850,70967,69029,67153,65335,63574,61869
17526,NR,Y_OPEN,T,UA,18720,11193,10365,8523,6839,6482,...,0,0,0,0,0,0,0,0,0,0
17527,NR,Y_OPEN,T,UK,0,0,13322,14469,14589,14407,...,427700,419900,408900,396300,400000,400000,400000,400000,300000,300000


In [59]:
df3 = df2.copy() # dont need totals

# make separate df for 'unkown' and 'open agegroups'
df_total = df3[df3["age"]=="TOTAL"].copy()
df_unk = df3[df3["age"]=="UNK"].copy()
df_yopen = df3[df3["age"]=="Y_OPEN"].copy()

# replace 'less than 1' string with 0
df3["age"].replace({"Y_LT1": 0}, inplace=True)

# subtract df's from main df
df3 = df3[~df3["age"].isin(df_total["age"])]
df3 = df3[~df3["age"].isin(df_unk["age"])]
df3 = df3[~df3["age"].isin(df_yopen["age"])]

# replace remaining ages with numeric values
df3["age"] = df3["age"].astype(str).str.replace("Y", "")
df3["age"] = pd.to_numeric(df3["age"], errors="coerce")
df3

,unit,age,sex,geo,2021,2020,2019,2018,2017,2016,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
348,NR,1,F,AD,0,0,240,0,0,0,...,0,0,0,0,0,0,0,0,0,0
349,NR,1,F,AL,13462,13638,14475,15170,15437,16812,...,0,0,0,0,0,0,0,0,0,0
350,NR,1,F,AM,17028,17217,17836,19007,19471,20124,...,0,0,0,0,0,0,0,0,0,0
351,NR,1,F,AT,41647,42255,42850,43340,41498,40860,...,60564,61362,61834,63929,64178,62999,61774,59624,58338,55948
352,NR,1,F,AZ,65513,64153,66803,73852,77132,78281,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17353,NR,0,T,SM,0,0,0,228,0,0,...,0,0,0,0,0,0,0,0,0,0
17354,NR,0,T,TR,1081967,1152226,1215156,1241005,1257621,1275773,...,903446,870966,839730,809687,780791,783259,785732,788217,790714,793219
17355,NR,0,T,UA,292037,307306,334177,361789,394626,409172,...,0,0,0,0,0,0,0,0,0,0
17356,NR,0,T,UK,0,0,733293,768431,772826,779610,...,0,0,0,0,0,0,0,0,0,0


In [95]:
bins_05y = [0, 4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, 69, 74, 79, 84, 89, 94, 99]
bins_10y = [0, 9, 19, 29, 39, 49, 59, 69, 79, 89, 99]
test = pd.DataFrame({"age": [0, 1, 2, 9, 10, 19, 20, 29, 99]})
test

,age
0,0
1,1
2,2
3,9
4,10
5,19
6,20
7,29
8,99


In [96]:
test["range"] = pd.cut(test["age"], bins_10y)
test

,age,range
0,0,NaN
1,1,"(0.0, 9.0]"
2,2,"(0.0, 9.0]"
3,9,"(0.0, 9.0]"
4,10,"(9.0, 19.0]"
5,19,"(9.0, 19.0]"
6,20,"(19.0, 29.0]"
7,29,"(19.0, 29.0]"
8,99,"(89.0, 99.0]"


In [97]:
test["range"] = pd.cut(test["age"], bins_10y, include_lowest=True, labels=[
    "00-09", "10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", 
    "80-89", "90-99"
    ])
test

,age,range
0,0,00-09
1,1,00-09
2,2,00-09
3,9,00-09
4,10,10-19
5,19,10-19
6,20,20-29
7,29,20-29
8,99,90-99


In [98]:
test["range"] = pd.cut(test["age"], bins_05y, include_lowest=True, labels=[
    "00-04", "05-09", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", 
    "40-44", "45-49", "50-54", "55-59", "60-64", "65-69", "70-74", "75-79",
    "80-84", "85-89", "90-94", "95-99"
    ])
test

,age,range
0,0,00-04
1,1,00-04
2,2,00-04
3,9,05-09
4,10,10-14
5,19,15-19
6,20,20-24
7,29,25-29
8,99,95-99
